<a href="https://colab.research.google.com/github/xuJ14/Math628Project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
rm -rf Math628Project

In [23]:
!git clone https://github.com/xuJ14/Math628Project

Cloning into 'Math628Project'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [26]:
# 查看工作路径
import os
 
print(os.getcwd())
 
 
# 修改工作路径
import os
from google.colab import drive

path = "/content/Math628Project"
os.chdir(path)

/content


In [35]:
import project
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


## no pca 2

In [79]:

Dow_estimators = dict()
Dow_estimators['rf']=RandomForestClassifier(max_features=1, n_estimators=500)
Dow_estimators['svc']=make_pipeline(StandardScaler(),SVC(C=10, gamma=0.1))
Dow_estimators['lgbm']=LGBMClassifier(learning_rate=0.5, max_depth=8, n_estimators=200, reg_lambda=10)
Dow_estimators['etc']=ExtraTreesClassifier(max_features=0.4)
Dow_estimators['xgb']=XGBClassifier(learning_rate=0.1,max_depth=8,n_estimators=200,reg_lambda=10)
Dow_estimators['clf']=StackingClassifier(estimators=list(Dow_estimators.items()),final_estimator=LogisticRegression(),cv=5)

In [80]:
SP500_estimators = dict()
SP500_estimators['rf']=RandomForestClassifier(max_features=0.2, n_estimators=200)
SP500_estimators['svc']=make_pipeline(StandardScaler(),SVC(C=10, gamma=0.1))
SP500_estimators['lgbm']=LGBMClassifier(learning_rate=0.5, max_depth=8, reg_lambda=1)
SP500_estimators['etc']=ExtraTreesClassifier(max_features=0.8, n_estimators=200)
SP500_estimators['xgb']=XGBClassifier(learning_rate=1,max_depth=8,n_estimators=200,reg_lambda=10)
SP500_estimators['clf']=StackingClassifier(estimators=list(SP500_estimators.items()),final_estimator=LogisticRegression(),cv=5)


In [81]:
Nasdaq_estimators = dict()
Nasdaq_estimators['rf']=RandomForestClassifier(n_estimators=200)
Nasdaq_estimators['svc']=make_pipeline(StandardScaler(),SVC(C=10, gamma=0.1))
Nasdaq_estimators['lgbm']=LGBMClassifier(learning_rate=0.5, max_depth=8, n_estimators=200, reg_lambda=0)
Nasdaq_estimators['etc']=ExtraTreesClassifier(max_features=0.8, n_estimators=200)
Nasdaq_estimators['xgb']=XGBClassifier(learning_rate=0.1,max_depth=8,n_estimators=200,reg_lambda=0.1)
Nasdaq_estimators['clf']=StackingClassifier(estimators=list(Nasdaq_estimators.items()),final_estimator=LogisticRegression(),cv=5)


In [82]:
%%capture
from sklearn import metrics
# evaluate
Model=dict()
name=['Dow','SP500','Nasdaq']
for i in name:
    data = pd.read_csv("{}_data_2.csv".format(i),index_col=0)
    data = data.dropna(axis=0,how='any')
    data.iloc[data.iloc[:,-1]==0,-1]=1
    # model data: 2006-01-01 to 2019-01-01
    model_data = data.iloc[pd.to_datetime(data.index)<pd.to_datetime('2019-01-01'),:]
    trace_data = data.iloc[pd.to_datetime(data.index)>=pd.to_datetime('2019-01-01'),:]
    X_train,X_test,y_train,y_test=train_test_split(model_data.iloc[:,:-1],model_data.iloc[:,-1][:,None],test_size=0.3,random_state=1)
    X_trace=trace_data.iloc[:,:-1]
    y_trace=trace_data.iloc[:,-1]
    
    #create the variable to save the data
    performance_matrix=pd.DataFrame(columns=['Accuracy','Precision','Recall','F1wght','F1micro'])
    X_train,X_test,X_trace,c=project.pca(X_train,X_test,X_trace,22)
    #calculate the score of eacy best model in train set.
    exec('best_estimators='+i+'_estimators')
    for name, model in best_estimators.items():
      print(name)
      model.fit(X_train,y_train)
      y_pred = model.predict(X_test)
      dicmet = {
          'Accuracy':metrics.accuracy_score(y_test,y_pred), 
          'Precision':metrics.precision_score(y_test,y_pred,average='weighted'),
          'Recall':metrics.recall_score(y_test,y_pred,average='weighted'),
          'F1wght':metrics.f1_score(y_test,y_pred,average='weighted'),
          'F1micro':metrics.f1_score(y_test,y_pred,average='micro')
          }
      performance_matrix.loc[name]=dicmet
    performance_matrix.to_csv(i+'_performance_pca_2.csv')


In [56]:
dicmet

{'Accuracy': 0.6588594704684317,
 'F1micro': 0.6588594704684317,
 'F1wght': 0.6506810744790203,
 'Precision': 0.6542688812358805,
 'Recall': 0.6588594704684317}

In [59]:
performance_matrix.loc[name]=dicmet

In [60]:
performance_matrix

,Accuracy,Precision,Recall,F1wght,F1micro,rf,svc,lgbm,etc,xgb,clf
Accuracy,NaN,NaN,NaN,NaN,NaN,Accuracy,Accuracy,Accuracy,Accuracy,Accuracy,Accuracy
Precision,NaN,NaN,NaN,NaN,NaN,Precision,Precision,Precision,Precision,Precision,Precision
Recall,NaN,NaN,NaN,NaN,NaN,Recall,Recall,Recall,Recall,Recall,Recall
F1wght,NaN,NaN,NaN,NaN,NaN,F1wght,F1wght,F1wght,F1wght,F1wght,F1wght
F1micro,NaN,NaN,NaN,NaN,NaN,F1micro,F1micro,F1micro,F1micro,F1micro,F1micro
clf,0.658859,0.654269,0.658859,0.650681,0.658859,NaN,NaN,NaN,NaN,NaN,NaN
